This is the main code for training a LSTM network and then saving the model, including some wrapper functions for pre-processing (mainly splitting) training data.

In [ ]:
'''imports'''
from __future__ import print_function
from __future__ import division

import keras, pickle
import pandas as pd
import numpy as np

from astropy.io import fits
from matplotlib import pyplot as plt

from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.callbacks import TerminateOnNaN, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM

from sklearn.metrics import mean_squared_error as mse

In [ ]:
def rmse(true, pred):
    MSE = mse(true, pred)
    return round(np.sqrt(MSE), 4)

In [ ]:
'''load training data'''
start_index = 0
n_frame = 30
xslos = fits.getdata('./datasets/trnslos_160k_open_withTT_gauss_0.1.fits')[:,start_index*72:(start_index+n_frame)*72]
# xslos = allslos[:,:36].copy()
# xslos = xslos.reshape((-1,n_frame*36))

In [ ]:
def _split(data, dim, split_point):
    """
    sub-function of train_test_split
    """
    if dim == 0:
        return (data[:split_point,:], data[split_point:,:])
    elif dim == 1:
        return (data[:,:split_point], data[:,split_point:])
    else:
        return (None, None)
    
def train_test_split(xslos, n_frame, test_ratio=0.0):  
    """
    This splits the whole data array as (x_train, y_train) and (x_test, y_test)
    """
    ntrn = int(round(xslos.shape[0] * (1 - test_ratio)))
    nx = (n_frame-1) * 72
    x, y = _split(xslos, 1, nx)
    x_train, x_test = _split(x, 0, ntrn)
    x_train = x_train.reshape((x_train.shape[0], n_frame-1, 72))
    x_test = x_test.reshape((x_test.shape[0], n_frame-1, 72))
    y_train, y_test = _split(y, 0, ntrn)
    return (x_train, y_train), (x_test, y_test)

In [ ]:
(x_train, y_train), (x_test, y_test) = train_test_split(xslos, n_frame)

In [ ]:
'''build the model - which is not optimal'''
in_out_neurons = 72  
hidden_neurons = 72

model = Sequential()
model.add(LSTM(hidden_neurons, return_sequences=True,input_shape=(None, in_out_neurons)))
model.add(LSTM(hidden_neurons, return_sequences=False))
model.add(Dense(in_out_neurons, input_dim=hidden_neurons))
model.compile(loss="mean_squared_error", optimizer=Adam(lr=0.001)) # or rmsprop

model.summary()

In [ ]:
'''training and saving the model'''
terminator = TerminateOnNaN()
lrreducer = ReduceLROnPlateau(verbose=1,factor=0.2,min_lr=1e-5)
history = model.fit(x_train, y_train, 
          batch_size=128, 
          epochs=20, 
          validation_split=0.1, 
          callbacks=[terminator, lrreducer])
print('Done!')

file_name = '1ly_72_72_20_128_100_withTT(lr=0.001)_gauss_0.01'
model.save('/home/xuewen/Dropbox/expout/'+file_name+'.h5')